In [ ]:
# | default_exp maxvit_3d

In [1]:
# | export

import torch
import numpy as np
from einops import rearrange, repeat
from einops.layers.torch import Rearrange, Reduce
from torch import nn, einsum
import torch.nn.functional as F
from torchview import draw_graph
import graphviz
import copy
graphviz.set_jupyter_format('png')

'svg'

In [2]:
# | export
# helpers
def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d

def cast_tuple(val, length = 1):
    return val if isinstance(val, tuple) else ((val,) * length)

# helper classes
class PreNormResidual(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x):
        return self.fn(self.norm(x)) + x

class MaxViT3DMLP(nn.Module):
    def __init__(self, dim, mult = 4, dropout = 0.):
        super().__init__()
        inner_dim = int(dim * mult)
        self.net = nn.Sequential(
            nn.Linear(dim, inner_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Residual(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

In [3]:
# | export
# MBConv
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool3d(1),    # input C x H x W x D --> C x 1 X 1 x 1  ONE value of each channel
            nn.Conv3d(in_channels, reduced_dim, kernel_size=1), # expansion
            nn.SiLU(), # activation
            nn.Conv3d(reduced_dim, in_channels, kernel_size=1), # brings it back
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        return x*self.se(x)

class CNNBlock3d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, padding = 0 , groups=1, act=True, bn=True, bias=False):
        super(CNNBlock3d, self).__init__()
        self.cnn = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding, groups=groups, bias=bias) #bias set to False as we are using BatchNorm
        
        # if groups = in_channels then it is for Depth wise convolutional; For each channel different Convolutional kernel
        # very limited change in loss but a very high decrease in number of paramteres
        # if groups = 1 : normal_conv kernel of size kernel_size**3

        self.bn = nn.BatchNorm3d(out_channels) if bn else nn.Identity() 
        self.silu = nn.SiLU() if act else nn.Identity() ## SiLU <--> Swish same Thing
        # 1 layer in MBConv doesn't have activation function
    
    def forward(self, x):
        out = self.cnn(x)
        out = self.bn(out)
        out = self.silu(out)
        return out

#dropout
class StochasticDepth(nn.Module):
    def __init__(self, survival_prob=0.8):
        super(StochasticDepth, self).__init__()
        self.survival_prob =survival_prob
        
    def forward(self, x): #form of dropout , randomly remove some layers not during testing
        if not self.training:
            return x
        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, 1, device= x.device) < self.survival_prob # maybe add 1 more here
        return torch.div(x, self.survival_prob) * binary_tensor
    
class MBConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, 
                 is_first = False,
                 expand_ratio = 6, 
                 reduction = 4, #squeeze excitation 1/4 = 0.25
                 survival_prob =0.8 # for stocastic depth
                 ):
        super(MBConv3d, self).__init__()
        
        stride = 2 if is_first else 1
        survival_prob = 0.8
        self.use_residual = True if not is_first else False
        hidden_dim = int(out_channels * expand_ratio)
        reduced_dim = int(in_channels/reduction)
        padding = padding
        
        ##expansion phase
        self.expand = nn.Identity() if (expand_ratio == 1) else CNNBlock3d(in_channels, hidden_dim, kernel_size = 1)
        
        ##Depthwise convolution phase
        self.depthwise_conv = CNNBlock3d(hidden_dim, hidden_dim,
                                        kernel_size = kernel_size, stride = stride, 
                                        padding = padding, groups = hidden_dim
                                       )
        
        # Squeeze Excitation phase
        self.se = SqueezeExcitation(hidden_dim, reduced_dim = reduced_dim)
        
        #output phase
        self.pointwise_conv = CNNBlock3d(hidden_dim, out_channels, kernel_size = 1, stride = 1, act = False, padding = 0)
        # add Sigmoid Activation as mentioned in the paper
        
        # drop connect
        self.drop_layers = StochasticDepth(survival_prob = survival_prob)

    
    def forward(self, x):
        residual = x
        x = self.expand(x)
        x = self.depthwise_conv(x)
        x = self.se(x)
        x = self.pointwise_conv(x)
        
        if self.use_residual:  #and self.depthwise_conv.stride[0] == 1:
            x = self.drop_layers(x)
            x += residual
        return x

In [4]:
# | export
# attention related classes
class MaxViT3DMHSA(nn.Module):
    def __init__(
        self,
        dim,
        dim_per_head = 32,
        dropout = 0.,
        window_size = (7,7,7)
    ):
        super().__init__()
        assert (dim % dim_per_head) == 0, 'dimension should be divisible by dimension per head'

        self.heads = dim // dim_per_head
        self.scale = dim_per_head ** -0.5

        self.to_qkv = nn.Linear(dim, dim * 3, bias = False)

        # self.attend = nn.Sequential(
        #     nn.Softmax(dim = -1),
        #     nn.Dropout(dropout)
        # )

        self.to_out = nn.Sequential(
            nn.Linear(dim, dim, bias = False),
            nn.Dropout(dropout)
        )
        
        # relative positional bias
        w1,w2,w3 = window_size
        self.rel_pos_bias = nn.Embedding((2 * w1 - 1) *(2 * w2 - 1)*(2 * w3 - 1), self.heads)
        pos1 = torch.arange(w1, dtype=torch.int32)
        pos2 = torch.arange(w2, dtype=torch.int32)
        pos3 = torch.arange(w3, dtype=torch.int32)
        # First we use the torch.arange and torch.meshgrid functions to generate the corresponding coordinates, [3,H,W,D] 
        # and then stack them up and expand them into a two-dimensional vector to get the absolute position index.
        grid = torch.stack(torch.meshgrid(pos1, pos2, pos3, indexing = 'ij'))
        grid = rearrange(grid, 'c i j k -> (i j k) c')
        # insert a dimension in the first dimension and the second dimension respectively, perform broadcast subtraction, and obtain the tensor of 3, whd*ww, whd*ww
        rel_pos = rearrange(grid, 'i ... -> i 1 ...') - rearrange(grid, 'j ... -> 1 j ...') 
        rel_pos[...,0] += w1 - 1
        rel_pos[...,1] += w2 - 1
        rel_pos[...,2] += w3 - 1
        # Do a multiplication operation to distinguish, sum up the last dimension, and expand it into a one-dimensional coordinate   a*x1 + b*x2 + c*x3  (a= hd b=d c =1) 
        rel_pos_indices = (rel_pos * torch.tensor([(2 *w2 - 1)*(2 *w3 - 1), (2 *w3 - 1), 1])).sum(dim = -1)
        
        # Register as a variable that does not participate in learning
        self.register_buffer('rel_pos_indices', rel_pos_indices, persistent = False)
        self.dropout_prob = dropout
               

    def forward(self, x):
        batch, height, width, depth, window_height, window_width, window_depth ,_ , h = *x.shape, self.heads
        # flatten
        x = rearrange(x, 'b x y z w1 w2 w3 d -> (b x y z) (w1 w2 w3) d')
        # project for queries, keys, values
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)

        # split heads
        q, k, v = map(lambda t: rearrange(t, 'b n (h d ) -> b h n d', h = h), (q, k, v))
        # scale
        q = q * self.scale

        # sim
        sim = einsum('b h i d, b h j d -> b h i j', q, k)

        # add positional bias
        bias = self.rel_pos_bias(self.rel_pos_indices)
        sim = sim + rearrange(bias, 'i j h -> h i j')

        # # attention
        # attn = self.attend(sim)

        context = F.scaled_dot_product_attention(
            q,
            k,
            v,
            dropout_p=self.dropout_prob,
            is_causal=False,
            scale=1.0,  # Already scaled the vectors
        )
        
        # # aggregate
        # out = einsum('b h i j, b h j d -> b h i d', attn, v)

        # merge heads
        out = rearrange(context, 'b h (w1 w2 w3) d -> b w1 w2 w3 (h d)', w1 = window_height, w2 = window_width, w3 = window_depth)

        # combine heads out
        out = self.to_out(out)
        return rearrange(out, '(b x y z) ... -> b x y z ...', x = height, y = width, z = depth)


In [5]:
# | export
class MaxViT3DStem0(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.conv_stem = nn.Sequential(
            nn.Conv3d(self.config['in_channels'], self.config['hidden_dim'], kernel_size=3, stride = 2, padding = 1),
            nn.Conv3d(self.config['hidden_dim'], self.config['out_channels'], 3, padding = 1)
        )
    
    def forward(self, x):
        return self.conv_stem(x)

In [6]:
test_stage_config ={ 
    "in_channels": 54,           # dimension of first layer, doubles every layer
    "hidden_dim": 98,            # dimension of attention heads, kept at 32 in paper`
    "out_channels": 54           # window size for block and grids
}

maxvit_block = MaxViT3DStem0(test_stage_config)
img = torch.randn(2, 54, 32, 32, 24)
preds = maxvit_block(img) 
print(preds.shape)

torch.Size([2, 54, 16, 16, 12])


In [7]:
# | export
class MaxViT3DBlock(nn.Module):
    def __init__(self, stage_config):
        super().__init__()
        
        self.stage_config = copy.deepcopy(stage_config)
        
        w1,w2,w3 = stage_config['window_size']
        stem_dim_in = stage_config['stem_dim_in']       
        dim = stage_config['dim']
        dim_per_head = stage_config['dim_per_head']
        dropout = stage_config['dropout']
        window_size = stage_config['window_size']
        expansion_rate = stage_config['expansion_rate']
        shrinkage_rate = stage_config['shrinkage_rate']
        is_first = stage_config['is_first']
        
        self.net = nn.Sequential(
            MBConv3d(
                in_channels = stem_dim_in if is_first else dim, 
                out_channels = dim,
                kernel_size=3 ,  
                padding=1 , 
                is_first=is_first,
                expand_ratio = expansion_rate if expansion_rate is not None else 4, 
                reduction = shrinkage_rate if shrinkage_rate is not None else 4, #squeeze excitation 1/4 = 0.25
                survival_prob = 1-dropout # for stocastic depth
            ),
            
            Rearrange('b d (x w1) (y w2) (z w3) -> b x y z w1 w2 w3 d', w1 = w1, w2 = w2, w3 = w3),  # block-like attention
            PreNormResidual(dim, MaxViT3DMHSA(dim = dim, dim_per_head = dim_per_head, dropout = dropout, window_size = window_size)),
            PreNormResidual(dim, MaxViT3DMLP(dim = dim, dropout = dropout)),
            Rearrange('b x y z w1 w2 w3 d -> b d (x w1) (y w2) (z w3)'),

            Rearrange('b d (w1 x) (w2 y) (w3 z) -> b x y z w1 w2 w3 d', w1 = w1, w2 = w2, w3 = w3),  # grid-like attention
            PreNormResidual(dim, MaxViT3DMHSA(dim = dim, dim_per_head = dim_per_head, dropout = dropout, window_size = window_size)),
            PreNormResidual(dim, MaxViT3DMLP(dim = dim, dropout = dropout)),
            Rearrange('b x y z w1 w2 w3 d -> b d (w1 x) (w2 y) (w3 z)'),
            )

    def forward(self, x):
        print(x.shape)
        x = self.net(x)
        return x

In [8]:
test_stage_config = {
    "stem_dim_in": 64,             # used in first block to downsample z,x,y
    "dim": 128,                    # dimension of whole layer, doubles every layer
    "is_first": True,
    "dim_per_head": 8,            # dimension of attention heads, kept at 32 in paper`
    "window_size": (4,4,4),        # window size for block and grids
    "dropout": 0.1,               # dropout
    "expansion_rate": None,        # squeeze and (excitation)
    "shrinkage_rate": None,        # (squeeze) and excitation
}

maxvit_block = MaxViT3DBlock(test_stage_config)
img = torch.randn(2, 64, 32, 32, 24)
preds = maxvit_block(img) 
print(preds.shape)

torch.Size([2, 64, 32, 32, 24])
torch.Size([2, 128, 16, 16, 12])


In [9]:
test_stage_config = {
    "stem_dim_in": 64,             # used in first block to downsample z,x,y
    "dim": 128,                    # dimension of whole layer, doubles every layer
    "is_first": False,
    "dim_per_head": 8,            # dimension of attention heads, kept at 32 in paper`
    "window_size": (4,4,4),        # window size for block and grids
    "dropout": 0.1,               # dropout
    "expansion_rate": None,        # squeeze and (excitation)
    "shrinkage_rate": None,        # (squeeze) and excitation
}

if __name__ == "__main__":
    maxvit_block = MaxViT3DBlock(test_stage_config)
    img = torch.randn(2, 128, 32, 32, 24)
    preds = maxvit_block(img) 
    print(preds.shape)

torch.Size([2, 128, 32, 32, 24])
torch.Size([2, 128, 32, 32, 24])


In [10]:
# batch_size = 2
# # device='meta' -> no memory is consumed for visualization
# model_graph = draw_graph(maxvit_block, input_size=(1, 256, 32, 32, 24), device='meta')
# model_graph.visual_graph

In [11]:
# | export

class MaxViT3DStem(nn.Module):
    def __init__(self, stage_config):
        super().__init__()

        self.config = copy.deepcopy(stage_config)
        self.first_block_config = copy.deepcopy(stage_config)

        self.first_block_config["is_first"] = True
        self.config["is_first"] = False
         
        self.stem = nn.ModuleList([])
        self.stem.append(MaxViT3DBlock(self.first_block_config))
        for i in range(self.config["num_maxvit_blocks"]-1):
            self.stem.append(MaxViT3DBlock(self.config))
            
    def forward(self, hidden_states: torch.Tensor):
        for layer in self.stem:
            hidden_states = layer(hidden_states)
        return hidden_states

In [12]:
test_stage_config = {
    "num_maxvit_blocks": 1,
    "stem_dim_in": 32,
    "dim": 64,                     # dimension of stem, doubles every stem
    "dim_per_head": 8,             # dimension of attention heads, kept at 32 in paper`
    "window_size": (4,4,4),        # window size for block and grids
    "dropout": 0.1,                # dropout
    "expansion_rate": None,        # squeeze and (excitation)
    "shrinkage_rate": None,        # (squeeze) and excitation  
}

test = MaxViT3DStem(test_stage_config)
img = torch.randn(2, 32, 32, 32, 24)
preds = test(img)
print(preds.shape)

torch.Size([2, 32, 32, 32, 24])
torch.Size([2, 64, 16, 16, 12])


In [13]:
# | export
class MaxViT3DEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.stems = nn.ModuleList([])
        
        self.stems.append(MaxViT3DStem0(config["stem0"]))
        
        for stage_config in config["stems"]:
            self.stems.append(MaxViT3DStem(stage_config))

    def forward(self, hidden_states: torch.Tensor):

        for stem in self.stems:
            hidden_states = stem(hidden_states)

        return hidden_states

In [18]:
# | export

test_encoder_config ={ 
    "stem0": {
        "in_channels": 1,         
        "hidden_dim": 32,         
        "out_channels": 32        
    },
    
    "stems": 
    [{
        "num_maxvit_blocks": 2,
        "stem_dim_in": 32,
        "dim": 64,                     # dimension of first layer, doubles every layer
        "dim_per_head": 8,             # dimension of attention heads, kept at 32 in paper`
        "window_size": (4,4,4),        # window size for block and grids
        "dropout": 0.1,                # dropout
        "expansion_rate": None,        # squeeze and (excitation)
        "shrinkage_rate": None,        # (squeeze) and excitation 
    },
    {
        "num_maxvit_blocks": 2,
        "stem_dim_in": 64,
        "dim": 128,                     # dimension of first layer, doubles every layer
        "dim_per_head": 8,             # dimension of attention heads, kept at 32 in paper`
        "window_size": (4,4,4),        # window size for block and grids
        "dropout": 0.1,                # dropout
        "expansion_rate": None,        # squeeze and (excitation)
        "shrinkage_rate": None,        # (squeeze) and excitation },    
    },
    {        
        "num_maxvit_blocks": 2,
        "stem_dim_in": 128,
        "dim": 256,                     # dimension of first layer, doubles every layer
        "dim_per_head": 8,             # dimension of attention heads, kept at 32 in paper`
        "window_size": (2,2,2),        # window size for block and grids
        "dropout": 0.1,                # dropout
        "expansion_rate": None,        # squeeze and (excitation)
        "shrinkage_rate": None,        # (squeeze) and excitation },}
    }]
}

In [19]:
encoder = MaxViT3DEncoder(test_encoder_config)

In [20]:
encoder(torch.randn(2, 1, 32, 256, 256))

torch.Size([2, 32, 16, 128, 128])
torch.Size([2, 64, 8, 64, 64])
torch.Size([2, 64, 8, 64, 64])
torch.Size([2, 128, 4, 32, 32])
torch.Size([2, 128, 4, 32, 32])
torch.Size([2, 256, 2, 16, 16])


tensor([[[[[-2.2867e+00,  1.1482e-02,  6.1498e-02,  ..., -1.7228e+00,
             8.2716e-01,  4.7021e-01],
           [-1.1849e-01, -1.3532e+00,  5.1944e-01,  ..., -6.4874e-01,
            -1.3123e+00,  1.0021e+00],
           [-1.2210e+00,  3.9233e-01,  4.0349e+00,  ...,  1.9783e+00,
            -9.1644e-01,  1.5266e+00],
           ...,
           [-7.1388e-01,  1.9040e+00, -2.7420e+00,  ..., -1.9899e+00,
             1.9202e+00, -1.0261e+00],
           [ 1.2139e+00,  3.5951e+00,  6.5220e-01,  ..., -7.0527e-01,
             9.6935e-01, -2.2203e+00],
           [-1.8401e+00, -7.3885e-01,  1.0114e+00,  ...,  1.6035e+00,
            -2.6275e+00, -4.1875e-01]],

          [[ 9.3094e-01, -1.0267e+00, -1.3423e+00,  ..., -4.0322e+00,
            -1.7851e+00, -4.7488e-01],
           [ 1.8620e-01,  2.3663e-01,  7.4904e-01,  ...,  5.9993e+00,
             5.1404e-01, -8.9890e-01],
           [ 4.4538e-01,  2.2139e+00, -2.0797e+00,  ...,  1.5005e+00,
             2.2805e+00, -1.2135e+00],
 

In [21]:
!nbdev_export